In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
dir0 = 'file:///home/cloudera/2.kkbox_churn/data01/'

In [4]:
infile = dir0 + '2.member_active_gaps'
rdd0_ = sc.textFile(infile)
rdd0_.take(5)

['msno,first_start_date,last_expire_date,gaps',
 '++7jKYbuIJPXry8Oh1NcEh9fCsqcQgUaaxXsgG15kMg=,2016-03-12,2016-03-22,',
 '+0RJtbyhoPAHPa+34MkYcE2Ox0cjMgJOTXMXVBYgkJE=,2016-01-13,2017-02-27,',
 '+3tRPOto/e58mdDA6oluFztHfwlqjJeNXb1vmYQjWfw=,2016-01-18,2016-02-17,',
 '+43UqHvqzu5STDVIuBLWryllqYYP4UrG9XyiajeGePk=,2015-12-08,2017-02-28,']

In [5]:
rdd0 = rdd0_.filter(lambda line: not line.startswith('msno,'))
rdd0.take(5)

['++7jKYbuIJPXry8Oh1NcEh9fCsqcQgUaaxXsgG15kMg=,2016-03-12,2016-03-22,',
 '+0RJtbyhoPAHPa+34MkYcE2Ox0cjMgJOTXMXVBYgkJE=,2016-01-13,2017-02-27,',
 '+3tRPOto/e58mdDA6oluFztHfwlqjJeNXb1vmYQjWfw=,2016-01-18,2016-02-17,',
 '+43UqHvqzu5STDVIuBLWryllqYYP4UrG9XyiajeGePk=,2015-12-08,2017-02-28,',
 '+4dMilwe46UsxpXXH0O7/mF3g+5fn4MAF2VTWPNzuoo=,2016-03-05,2016-04-04,']

In [13]:
Nmem = rdd0.count()
print(Nmem)

649618


# ending gap
(if last expiration date < 2017-02-28)

In [6]:
from datetime import datetime, timedelta

def date_add_one(date_str):
    d = datetime.strptime(date_str, '%Y-%m-%d') + timedelta(days = 1)
    return d.strftime('%Y-%m-%d')

def get_ending_gap(rec):
    '''
    Returns a list (either empty or containing 1 tuple).
    '''
    fields = rec.split(',')
    
    if fields[2] >= '2017-02-28':
        return []
    
    begin = date_add_one(fields[2])
    end = '2017-02-28'
    return [(fields[0], begin+'_'+end)]

In [7]:
rdd_ending_gap = rdd0.flatMap(get_ending_gap)
rdd_ending_gap.take(5)

[('++7jKYbuIJPXry8Oh1NcEh9fCsqcQgUaaxXsgG15kMg=', '2016-03-23_2017-02-28'),
 ('+0RJtbyhoPAHPa+34MkYcE2Ox0cjMgJOTXMXVBYgkJE=', '2017-02-28_2017-02-28'),
 ('+3tRPOto/e58mdDA6oluFztHfwlqjJeNXb1vmYQjWfw=', '2016-02-18_2017-02-28'),
 ('+4dMilwe46UsxpXXH0O7/mF3g+5fn4MAF2VTWPNzuoo=', '2016-04-05_2017-02-28'),
 ('+5p5fm9oHH8ukkNG1b+wuhWKCM6Ykmk2sRZnQ6MKwoA=', '2016-07-09_2017-02-28')]

In [20]:
rdd_ending_gap.count()

542913

# middle gaps

In [25]:
rdd0.filter(lambda rec: len(rec)>90).take(1)

['+R423gwTlAs/eYBATR7PO6ZNh7moqhzA8xdDycFoBzA=,2016-11-08,2017-03-23,2016-11-16_2016-11-21|2017-01-23_2017-02-20']

In [8]:
def get_middle_gaps(rec):
    '''
    Returns a list.
    '''
    fields = rec.split(',')
    if len(fields[3]) == 0:
        return []
    
    # fields[3] is like '2016-11-16_2016-11-21|2017-01-23_2017-02-20'
    
    result = []
    gaps = fields[3].split('|')
    for gap in gaps:
        # `gap` is like '2016-11-16_2016-11-21'
        result.append((fields[0], gap))
        
    return result

In [9]:
rdd_mid_gaps = rdd0.flatMap(get_middle_gaps)
rdd_mid_gaps.take(10)

[('+BauTESekzh7dOwBqraMw7OFo8WDD5nSCx5lRV3+f20=', '2016-03-24_2016-03-25'),
 ('+MjSIVqd6qvlkKaP36fgvZYpm7Ohuxdu6Adpu/1AltE=', '2016-01-08_2016-02-04'),
 ('+R423gwTlAs/eYBATR7PO6ZNh7moqhzA8xdDycFoBzA=', '2016-11-16_2016-11-21'),
 ('+R423gwTlAs/eYBATR7PO6ZNh7moqhzA8xdDycFoBzA=', '2017-01-23_2017-02-20'),
 ('+Ub6MOxsZzLGHQY7PboV2+WynMkYNw4EMb/6OBm1hAI=', '2016-07-22_2016-07-30'),
 ('+Vzca1ck/BX03oBYaZ6itKg0TRbjlTguSvBSoklmQdM=', '2016-01-18_2016-01-18'),
 ('+Vzca1ck/BX03oBYaZ6itKg0TRbjlTguSvBSoklmQdM=', '2016-02-19_2016-02-19'),
 ('+Vzca1ck/BX03oBYaZ6itKg0TRbjlTguSvBSoklmQdM=', '2016-04-21_2016-04-23'),
 ('+Vzca1ck/BX03oBYaZ6itKg0TRbjlTguSvBSoklmQdM=', '2016-06-25_2016-06-25'),
 ('+Vzca1ck/BX03oBYaZ6itKg0TRbjlTguSvBSoklmQdM=', '2016-07-27_2016-07-28')]

# union

In [10]:
rdd_all_gaps = rdd_mid_gaps.union(rdd_ending_gap)

In [32]:
rdd_all_gaps.count()

1168448

# convert date to integer

In [28]:
rdd_all_gaps.take(3)

[('+BauTESekzh7dOwBqraMw7OFo8WDD5nSCx5lRV3+f20=', '2016-03-24_2016-03-25'),
 ('+MjSIVqd6qvlkKaP36fgvZYpm7Ohuxdu6Adpu/1AltE=', '2016-01-08_2016-02-04'),
 ('+R423gwTlAs/eYBATR7PO6ZNh7moqhzA8xdDycFoBzA=', '2016-11-16_2016-11-21')]

In [29]:
from datetime import datetime

date0 = datetime(2015, 11, 1)

def date_to_int(date_str):
    global date0
    d = datetime.strptime(date_str, '%Y-%m-%d')
    return (d - date0).days

def gap_shorten(gap_str):
    # gap_str is like '2016-03-24_2016-03-25'
    begin_end = gap_str.split('_')
    return str(date_to_int(begin_end[0]))+'-'+str(date_to_int(begin_end[1]))

In [31]:
rdd_all_gaps_1 = rdd_all_gaps.map(lambda rec: (rec[0], gap_shorten(rec[1])))
rdd_all_gaps_1.take(3)

[('+BauTESekzh7dOwBqraMw7OFo8WDD5nSCx5lRV3+f20=', '144-145'),
 ('+MjSIVqd6qvlkKaP36fgvZYpm7Ohuxdu6Adpu/1AltE=', '68-95'),
 ('+R423gwTlAs/eYBATR7PO6ZNh7moqhzA8xdDycFoBzA=', '381-386')]

# group by member

In [35]:
from pyspark.sql import Row
df2 = rdd_all_gaps_1.toDF() \
        .selectExpr('_1 AS msno', '_2 AS gap')
df2.show(5)

+--------------------+-------+
|                msno|    gap|
+--------------------+-------+
|+BauTESekzh7dOwBq...|144-145|
|+MjSIVqd6qvlkKaP3...|  68-95|
|+R423gwTlAs/eYBAT...|381-386|
|+R423gwTlAs/eYBAT...|449-477|
|+Ub6MOxsZzLGHQY7P...|264-272|
+--------------------+-------+
only showing top 5 rows



In [37]:
from pyspark.sql.functions import col, concat_ws, collect_list

df3 = df2.groupBy('msno') \
         .agg(concat_ws('|', collect_list('gap')).alias('gaps'))
df3.show(25)

+--------------------+---------------+
|                msno|           gaps|
+--------------------+---------------+
|++7jKYbuIJPXry8Oh...|        143-485|
|+0RJtbyhoPAHPa+34...|        485-485|
|+3tRPOto/e58mdDA6...|        109-485|
|+4dMilwe46UsxpXXH...|        156-485|
|+5p5fm9oHH8ukkNG1...|        251-485|
|+7LsJK3q0POdtDWvw...|        244-485|
|+7f07tkg6Q6sPq1MX...|        390-485|
|+8suRKxAwKNflQAfd...|        378-485|
|+98dadrcy1re8eJQo...|        366-485|
|+9X+kOvArYU6AYtdY...|        386-485|
|+BEFhhOsyuiTg14Kq...|         44-485|
|+BauTESekzh7dOwBq...|177-485|144-145|
|+CP9ZYRo9JnR0ICPZ...|        385-485|
|+CbFktd/ap/Zxkfla...|        399-485|
|+J87iIu2aFd2ZXf6g...|        243-485|
|+JUv12RygY6bgv4Vo...|        398-485|
|+JnhJcr4oD5Z0kX3E...|         32-485|
|+KOCWQtfYg/AzdZxZ...|        395-485|
|+KeVLXI9EeGG/mPEi...|        121-485|
|+MjSIVqd6qvlkKaP3...|  107-485|68-95|
|+NaRi/wkOw9wtDFHM...|        351-485|
|+PSMpTcFNjoswdFWH...|        120-485|
|+QWX04qIeP5V7H5UU...|   

# output

In [38]:
outfile = dir0 + '3.member_inactive-periods_for_check-w-userlogs'
df3.coalesce(1).write.format('csv').option('header', 'true').save(outfile)